In [5]:
# Importar librerías para tratamiento de datos
import pandas as pd  

# Importar librerías para web scraping y manipulación de datos
from bs4 import BeautifulSoup
import requests
import re

# Importar librerías para automatización de navegadores web con Selenium
from selenium import webdriver  
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select  
# Importar librerías para pausar la ejecución

from time import sleep  

# Configuraciones
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import warnings
warnings.filterwarnings('ignore')

In [6]:
df_pelis = pd.read_csv("COPIA SEGURIDAD/union_peliculas.csv")

In [7]:
df_pelis.head()

,Unnamed: 0,Tipo,Título,Año de estreno,Mes de estreno,ID
0,0,Movie,Istota,2000,NaN,tt0102362
1,1,Movie,For the Cause,2000,5.0,tt0113092
2,2,Movie,Florentino y el diablo,2000,4.0,tt0113086
3,3,Movie,Karobaar: The Business of Love,2000,9.0,tt0116748
4,4,Movie,Gang,2000,4.0,tt0116391


In [8]:
lista_ID = df_pelis["ID"].to_list()

In [9]:
driver = webdriver.Chrome()

url = "https://www.imdb.com/?ref_=nv_home"

driver.get(url)

driver.maximize_window()

sleep(5)

# Aceptar las cookies PAG IMDB
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

sleep(5)


## MODIFICAR, tiene que ser una lista de tuplas
diccionario_peliculas = {"Puntuacion_IMDB":[], "Dirección": [],"Guionistas":[], "Argumento":[], "Duración":[], "Nombre de la película": []}

lista_ID

for pelicula in lista_ID[9:12]:
    
    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#suggestion-search').send_keys(pelicula, Keys.ENTER)

    sleep(5)

    # Entramos a la pagina de la peli
    # driver.find_element("css selector", "#__next > main > div.ipc-page-content-container.ipc-page-content-container--full.sc-a8b15070-0.iMRIeM > div.ipc-page-content-container.ipc-page-content-container--center > section > div > div.ipc-page-grid__item.ipc-page-grid__item--span-2 > section:nth-child(3) > div.sc-17bafbdb-2.iUyFfD > ul > li:nth-child(1) > div.ipc-metadata-list-summary-item__c > div > a").click()

    # sleep(5)

# Seleccionar nombre de la pelicula
    try:
        diccionario_peliculas["Nombre de la película"].append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-69e49b85-0.jqlHBQ > h1 > span").text)
    except:
        print("error nombre peli")
# Seleccionar puntuacion 
    try: 
        diccionario_peliculas["Puntuacion_IMDB"].append(driver.find_element("xpath",'//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div[2]/div/div[1]/a/span/div/div[2]/div[1]/span[1]').text)
    except:  
        print("error PUNTUACION")

# Seleccionar director
    try:  
        diccionario_peliculas["Dirección"].append(driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div > ul > li > a").text)
    except:
        print("error DIRECTOR")  

# Seleccionar guionistas        
    try:
        diccionario_peliculas["Guionistas"].append(driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul > li > a").text)
    except:
        diccionario_peliculas["Guionistas"].append("Desconocido")
    
    sleep(5)
# Seleccionar ARGUMENTO
    try:     
        driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-66ec1b32-0.kbbKze > div > div:nth-child(2) > button").click()
        driver.find_element("css selector", "body > div.ipc-promptable-base.ipc-promptable-dialog.sc-5356e79c-1.fpIaNP.all-topics-prompt.enter-done > div.ipc-promptable-base__panel.all-topics-prompt__panel > div > div.ipc-promptable-base__auto-focus > div > div.ipc-promptable-base__content > div > div.sc-26f539cf-0.hEkldg > div:nth-child(1) > div > div:nth-child(2) > a").click()
        diccionario_peliculas["Argumento"].append(driver.find_element("css selector", "#po1590418 > div > div > div").text)
        driver.back() 
    
    except:
        try:
            diccionario_peliculas["Argumento"].append(driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-4.yjUiO > div.sc-491663c0-6.lnlBxO > div.sc-491663c0-10.rbXFE > section > p").text)
   
        except:
            diccionario_peliculas["Argumento"].append("Desconocido")

# Seleccionar DURACION        
    try:   
        diccionario_peliculas["Duración"].append(driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(4) > section > section > div.sc-491663c0-3.bdjVSf > div.sc-69e49b85-0.jqlHBQ > ul").text)
    except:
            print("error Duracion") 



ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=122.0.6261.57)
Stacktrace:
	GetHandleVerifier [0x00007FF65E1D4C82+3505170]
	(No symbol) [0x00007FF65DE00852]
	(No symbol) [0x00007FF65DCB3FFD]
	(No symbol) [0x00007FF65DCF4BC7]
	(No symbol) [0x00007FF65DCF3842]
	(No symbol) [0x00007FF65DD1BE8A]
	(No symbol) [0x00007FF65DCEF27A]
	(No symbol) [0x00007FF65DD1C0A0]
	(No symbol) [0x00007FF65DD383B2]
	(No symbol) [0x00007FF65DD1BC33]
	(No symbol) [0x00007FF65DCED618]
	(No symbol) [0x00007FF65DCEE6B1]
	GetHandleVerifier [0x00007FF65E2067DD+3708781]
	GetHandleVerifier [0x00007FF65E25FC5D+4074477]
	GetHandleVerifier [0x00007FF65E257DDF+4042095]
	GetHandleVerifier [0x00007FF65DF2A136+708806]
	(No symbol) [0x00007FF65DE0CB0F]
	(No symbol) [0x00007FF65DE07D14]
	(No symbol) [0x00007FF65DE07E6C]
	(No symbol) [0x00007FF65DDF79A4]
	BaseThreadInitThunk [0x00007FF9DFB4257D+29]
	RtlUserThreadStart [0x00007FF9E1D0AA58+40]


In [35]:
diccionario_peliculas

{'Puntuacion_IMDB': ['5,4', '5,0', '6,6'],
 'Dirección': ['Bruno Nuytten', 'Adrian Fulle', 'Dorian Ramírez'],
 'Guionistas': ['Bruno Nuytten', 'Adrian Fulle', 'Dorian Ramírez'],
 'Argumento': ['Desconocido', 'Desconocido', 'Desconocido'],
 'Duración': ['2000\n1h 44min', '2000\nR\n1h 26min', '2000\n1h 15min'],
 'Nombre de la película': ['Passionnément',
  'Love 101',
  'Pushing the Envelope']}

In [63]:
driver = webdriver.Chrome()

url= "https://www.google.es/"

driver.get(url)

driver.maximize_window()

# Aceptar las cookies de GOOGLE
driver.find_element("css selector", "#L2AGLb > div").click()


sleep(5)

## MODIFICAR, tiene que ser una lista de tuplas
diccionario_peliculas = {"Rotten Tomatoes":[]}

lista_titulos

for pelicula in lista_titulos[:10]:
    
    busqueda = f"puntuacion Rotten Tomatoes de la {pelicula}"


    sleep(5)


    #Accedemos al buscador e introduccimos la peli para buscar PAG IMDB
    driver.find_element("css selector",'#APjFqb').send_keys(busqueda, Keys.ENTER)


    sleep(5)
    try:
        diccionario_peliculas["Rotten Tomatoes"].append(driver.find_element("css selector","#rso > div:nth-child(3) > div > div > div > div:nth-child(3) > div > span:nth-child(2)").text)
    except:
        puntuacion_completa= (driver.find_element("css selector",'#rso > div:nth-child(1) > div > div > div:nth-child(3) > div').text)
        puntuacion = re.search(r'\b(\d+)%', puntuacion_completa)
        diccionario_peliculas["Rotten Tomatoes"].append(puntuacion.group())




NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#rso > div:nth-child(1) > div > div > div:nth-child(3) > div"}
  (Session info: chrome=121.0.6167.185); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF63BFD7012+3522402]
	(No symbol) [0x00007FF63BBF8352]
	(No symbol) [0x00007FF63BAA5ABB]
	(No symbol) [0x00007FF63BAEBF0E]
	(No symbol) [0x00007FF63BAEC08C]
	(No symbol) [0x00007FF63BB2E437]
	(No symbol) [0x00007FF63BB0F09F]
	(No symbol) [0x00007FF63BB2BDA3]
	(No symbol) [0x00007FF63BB0EE03]
	(No symbol) [0x00007FF63BADF4D4]
	(No symbol) [0x00007FF63BAE05F1]
	GetHandleVerifier [0x00007FF63C009B9D+3730157]
	GetHandleVerifier [0x00007FF63C05F02D+4079485]
	GetHandleVerifier [0x00007FF63C0575D3+4048163]
	GetHandleVerifier [0x00007FF63BD2A649+718233]
	(No symbol) [0x00007FF63BC04A3F]
	(No symbol) 

In [62]:
print(diccionario_peliculas)

{'Rotten Tomatoes': ['80%']}
